In [1]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")

import oge.download_data as download_data
import oge.load_data as load_data
from oge.column_checks import get_dtypes
from oge.filepaths import *
import oge.impute_hourly_profiles as impute_hourly_profiles
import oge.data_cleaning as data_cleaning
import oge.output_data as output_data
import oge.emissions as emissions
import oge.validation as validation
import oge.gross_to_net_generation as gross_to_net_generation
import oge.eia930 as eia930
from oge.logging_util import get_logger, configure_root_logger

configure_root_logger()
logger = get_logger("test")


year = 2022
path_prefix = f"{year}/"

In [22]:
# create a test output file
os.environ["OGE_DATA_STORE"] = "local"
print(outputs_folder())
# read from local
hourly_profiles = pd.read_csv(outputs_folder("2020/subplant_crosswalk_2020.csv"))
hourly_profiles.to_csv(
    outputs_folder("2020/subplant_crosswalk_2020.csv.zip"),
    index=False,
    compression="zip",
)

C:/Users/greg.miller/open_grid_emissions_data/outputs/


In [23]:
# test reading locally
print(outputs_folder())
test_local = pd.read_csv(outputs_folder("2020/subplant_crosswalk_2020.csv.zip"))
test_local.head()

C:/Users/greg.miller/open_grid_emissions_data/outputs/


,plant_id_epa,emissions_unit_id_epa,plant_id_eia,generator_id,subplant_id,unit_id_pudl,generator_operating_date,generator_retirement_date,current_planned_generator_operating_date,prime_mover_code
0,NaN,NaN,1,1,1,NaN,2000-12-01,NaN,NaN,IC
1,NaN,NaN,1,2,2,NaN,2000-12-01,NaN,NaN,IC
2,NaN,NaN,1,5,3,NaN,2000-12-01,NaN,NaN,IC
3,NaN,NaN,1,3,4,NaN,2010-12-01,NaN,NaN,IC
4,NaN,NaN,1,WT1,5,NaN,2011-10-01,NaN,NaN,WT


In [24]:
# test reading from s3 after uploading
os.environ["OGE_DATA_STORE"] = "s3"
print(outputs_folder())
test_s3 = pd.read_csv(outputs_folder("2020/subplant_crosswalk_2020.csv.zip"))
test_s3.head()

s3://open-grid-emissions/open_grid_emissions_data/outputs/


,plant_id_epa,emissions_unit_id_epa,plant_id_eia,generator_id,subplant_id,unit_id_pudl,generator_operating_date,generator_retirement_date,current_planned_generator_operating_date,prime_mover_code
0,NaN,NaN,1,1,1,NaN,2000-12-01,NaN,NaN,IC
1,NaN,NaN,1,2,2,NaN,2000-12-01,NaN,NaN,IC
2,NaN,NaN,1,5,3,NaN,2000-12-01,NaN,NaN,IC
3,NaN,NaN,1,3,4,NaN,2010-12-01,NaN,NaN,IC
4,NaN,NaN,1,WT1,5,NaN,2011-10-01,NaN,NaN,WT
